In [1]:
import copy
import time
import tqdm

from application.aws import AwsAPI
from application.game import Game
from application.utils import (
    get_game_events,
    dump_message_to_file,
    generate_batch_sizes,
)
from typing import Generator

# Bedrock knowledge bases test

This is the same test as in `game_events_realistic.ipynb`, but game state data is stored inside the knowledge base instead of prompts.

In [2]:
game_events = get_game_events("data/events.csv")

In [3]:
form_prompt = lambda new_events_num: (
    "Imagine you're commenting the battle royale game match. "
    "We have a knowledge base where we store information about the game events. "
    f"Recently, {new_events_num} new events were added to the game.json file, you can decide either to comment on all of them or comment the last of them while also keeping in mind other events. "
    "You're not supposed to always use each field for the comment, but you can use them if you think they're relevant. "
    "Try to also remember previous events and if you see some patterns feel free to voice them. "
    "Please comment on the recent events using 3 senteces max. "
)

In [4]:
def _get_sentences(new_events_num: int, aws: AwsAPI, model_id: str, temperature: float) -> Generator[str, None, None]:
    model = aws.models_mapping[model_id]

    yield from aws.get_streamed_response_rag(model, form_prompt(new_events_num), time.time())

In [5]:
game = Game(game_events)
winner = game.create_players_tree()
num_events_total = game.num_players_total - 1
batch_sizes = generate_batch_sizes(num_events_total)


def run_dialogue(model_id, region, temperature=0.9):
    aws = AwsAPI(region)
    events_nums = copy.copy(batch_sizes)
    filename = f"output/text/RAG/{int(time.time())}-{model_id}.txt"

    game.start()

    elapsed_total, events_num, new_events_num = 0, 0, 0
    game_state = []
    for event in tqdm.tqdm(game.get_events(winner), total=num_events_total):
        game_state.append(event)
        if events_num < events_nums[-1]:
            events_num += 1
            continue
        else:
            new_events_num = events_num
            events_nums.pop()
            events_num = 1

        start_time = time.time()
        aws.update_knowledge_base(game_state)
        
        response = []
        try:
            for sentence in _get_sentences(new_events_num, aws, model_id, temperature):
                response.append(sentence)
        except KeyboardInterrupt:
            break
        elapsed_total += time.time() - start_time

        dump_message_to_file("Assistant", "".join(response), filename)

    print(f"Avg. time per events batch: {elapsed_total / len(batch_sizes):.2f}s")


print("Testing Nova Pro with Knowledge Base...")
run_dialogue("amazon.nova-pro-v1:0", "us-east-1")

Testing Nova Pro with Knowledge Base...


100%|██████████| 130/130 [03:18<00:00,  1.52s/it]

Avg. time per events batch: 3.81s
